# wtbからcsvの学習データを作成

今回は2010から2020のデータを用いる

指手が範囲外のものは除去

パス、60手以内に終わった試合がどうなっているのか未確認

パス自体は学習データ作成時にそこまで影響はないはず、問題はその時の指手がどのようになっているのか

盤面の最初の方は同じものがいっぱいあるはずなので、重みをつけて（後半の盤面情報が多くなるように）学習データを作成するのもあり

今回は指手すべてを学習データにした、（負けやよくないさしてもあるはず）、よりよい学習データのためには考慮すべき

## wtbから指手のcsvに変換

wthor_to_csv.pyを使って変換

In [4]:
!python wthor_to_csv.py WTH_2020.wtb WTH_2019.wtb WTH_2018.wtb WTH_2017.wtb WTH_2016.wtb WTH_2015.wtb WTH_2014.wtb WTH_2013.wtb WTH_2012.wtb WTH_2011.wtb WTH_2010.wtb 

## 指手のcsvから学習用（データ、ラベル）に変換

範囲外のものも含む（ラベルが0-63以外のもの）

手順


1.   指手のcsvをpandasで読み取る
2.   リストに変換
3.   指手をインデックスに変換（ラベルデータにもなる）
4.   指手から順番に盤面を作成する関数を作成


In [5]:
import numpy as np
import pandas as pd

In [6]:
dataset = pd.read_csv('/content/dataset.csv', index_col=0, header=0)

In [7]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,e6,f4,c3,c4,f3,d6,c5,e3,f6,f5,g5,c6,d3,b5,a6,a4,d7,c7,g4,d8,b8,e7,e8,c8,b4,a8,b7,f8,b6,c2,d2,a7,c1,a5,b3,a3,f7,g8,h8,a2,g6,h6,e1,e2,f1,f2,g7,h7,h5,h4,h3,h2,g3,g1,g2,h1,b2,a1,b1,d1
1,e6,f4,c3,c4,d3,c6,e3,f3,d6,c7,b4,c5,f5,g5,f6,d7,g6,g4,b5,e7,f7,f8,h5,h4,h3,g3,h6,e2,h2,b6,d2,f2,b8,d8,a6,a3,a5,d1,c2,b3,b1,c1,e1,g1,e8,c8,g8,a8,g7,a7,b7,a4,b2,a1,f1,a2,h1,g2,h7,h8
2,e6,f6,f5,d6,c5,e3,d7,f4,d3,e7,g5,c6,g4,c4,c7,d8,b4,h4,h6,h3,g6,h5,f8,h7,f7,f3,b6,c2,c8,e8,e2,c3,c1,b5,a5,b3,d2,a3,g3,h2,a4,a6,b7,a8,a7,b8,g7,h8,a2,g8,a1,b2,b1,d1,f2,g2,f1,e1,h1,g1
3,e6,f6,f5,d6,d7,g5,g6,e7,c5,c4,e8,f7,c6,d8,c7,e3,f4,h6,c8,b5,f3,d3,g4,b6,e2,h3,f8,f1,e1,f2,d2,g3,b3,b4,c3,d1,c1,c2,g1,a3,g2,g7,b1,b7,a8,b8,a5,h1,h8,g8,h7,a1,a2,a4,b2,h5,h4,h2,a7,a6
4,e6,f4,e3,f2,e2,f6,f5,d6,e7,f8,e8,d8,f3,f7,d7,g5,f1,c8,g6,h6,h5,h4,c7,g4,g3,d2,c6,d1,d3,b8,c1,b1,g7,h3,b7,h2,c5,h7,g2,c3,c4,h1,g1,e1,c2,a8,a7,a6,b5,b6,b4,a5,a4,a3,b3,a2,b2,a1,g8,h8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21513,e6,f4,c3,c4,d3,d6,f6,c6,d7,e7,c5,c8,f5,e3,c7,g4,e8,b5,f8,b4,a4,a5,a6,f7,a3,b6,g5,d2,a7,b8,d1,h6,g6,h7,h3,h4,g3,f2,f3,g2,h5,h2,f1,e2,e1,c1,h1,g1,b1,g8,d8,b3,h8,b7,a2,b2,c2,g7,a8,a1
21514,e6,f6,f5,d6,c5,e3,d3,g5,f3,b5,c6,c4,c3,b4,a5,a3,b6,b3,a4,a6,f7,f4,g6,e7,e8,e2,e1,f1,g1,g4,f2,g3,c7,h6,d7,d2,h3,h4,g2,f8,b7,d8,h5,b8,c8,h1,h7,d1,a8,g7,g8,c1,c2,a7,b1,h8,b2,a2,a1,h2
21515,e6,f4,c3,c4,d3,d6,c5,e7,f3,e3,g4,g5,f5,d2,f2,b5,b4,c2,b3,a3,d1,e1,f1,c1,b1,f6,e2,h4,h3,h2,e8,f7,c7,d8,c8,d7,g3,g2,h6,h5,g6,f8,g8,c6,a5,b6,a6,a7,a8,b8,b7,h7,h1,h8,g7,g1,a1,a2,b2,a4
21516,e6,f4,c3,c4,d3,d6,e3,d2,e2,c2,g4,f3,c6,f6,g6,f5,g3,g5,h5,f2,c5,h3,f1,h4,h2,h7,f7,e7,d8,f8,d7,e8,g8,b5,e1,b7,d1,g1,h1,g2,a8,c1,b6,b4,c7,h6,h8,g7,b1,b2,a5,b3,a4,a2,a1,a6,a3,a7,c8,b8


In [10]:
records_list = dataset.values.tolist()

In [11]:
# AlphaZeroの本から、少し変更（return_board, 8×8に対応）
import random
import math

# ゲーム状態
class State:
    # 初期化
    def __init__(self, pieces=None, enemy_pieces=None, depth=0):
        # 方向定数
        self.dxy = ((1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1))

        # 連続パスによる終了
        self.pass_end = False

        # 石の配置
        self.pieces = pieces
        self.enemy_pieces = enemy_pieces
        self.depth = depth

        # 石の初期配置
        if pieces == None or enemy_pieces == None:
            self.pieces = [0] * 64
            self.pieces[28] = self.pieces[35] = 1
            self.enemy_pieces = [0] * 64
            self.enemy_pieces[27] = self.enemy_pieces[36] = 1

    # 石の数の取得
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count +=  1
        return count

    # 負けかどうか
    def is_lose(self):
        return self.is_done() and self.piece_count(self.pieces) < self.piece_count(self.enemy_pieces)

    # 引き分けかどうか
    def is_draw(self):
        return self.is_done() and self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # ゲーム終了かどうか
    def is_done(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 64 or self.pass_end

    # 次の状態の取得
    def next(self, action):
        state = State(self.pieces.copy(), self.enemy_pieces.copy(), self.depth+1)
        if action != 64:
            state.is_legal_action_xy(action%8, int(action/8), True)
        w = state.pieces
        state.pieces = state.enemy_pieces
        state.enemy_pieces = w

        # 2回連続パス判定
        if action == 64 and state.legal_actions() == [64]:
            state.pass_end = True
        return state

    # 合法手のリストの取得
    def legal_actions(self):
        actions = []
        for j in range(0,8):
            for i in range(0,8):
                if self.is_legal_action_xy(i, j):
                    actions.append(i+j*8)
        if len(actions) == 0:
            actions.append(64) # パス
        return actions

    # 任意のマスが合法手かどうか
    def is_legal_action_xy(self, x, y, flip=False):
        # 任意のマスの任意の方向が合法手かどうか
        def is_legal_action_xy_dxy(x, y, dx, dy):
            # １つ目 相手の石
            x, y = x+dx, y+dy
            if y < 0 or 7 < y or x < 0 or 7 < x or \
                self.enemy_pieces[x+y*8] != 1:
                return False

            # 2つ目以降
            for j in range(8):
                # 空
                if y < 0 or 7 < y or x < 0 or 7 < x or \
                    (self.enemy_pieces[x+y*8] == 0 and self.pieces[x+y*8] == 0):
                    return False

                # 自分の石
                if self.pieces[x+y*8] == 1:
                    # 反転
                    if flip:
                        for i in range(8):
                            x, y = x-dx, y-dy
                            if self.pieces[x+y*8] == 1:
                                return True
                            self.pieces[x+y*8] = 1
                            self.enemy_pieces[x+y*8] = 0
                    return True
                # 相手の石
                x, y = x+dx, y+dy
            return False

        # 空きなし
        if self.enemy_pieces[x+y*8] == 1 or self.pieces[x+y*8] == 1:
            return False

        # 石を置く
        if flip:
            self.pieces[x+y*8] = 1

        # 任意の位置が合法手かどうか
        flag = False
        for dx, dy in self.dxy:
            if is_legal_action_xy_dxy(x, y, dx, dy):
                flag = True
        return flag

    # 先手かどうか
    def is_first_player(self):
        return self.depth%2 == 0

    # 文字列表示
    def return_board(self):
        # ox = ('6', '9') if self.is_first_player() else ('9', '6')
        board = []
        for i in range(64):
            if self.pieces[i] == 1:
                board.append(1)
            elif self.enemy_pieces[i] == 1:
                board.append(-1)
            else:
                board.append(0)
            # if i % 8 == 7:
            #     str += '\n'
        return board

In [12]:
#指手をインデックスの数字に変換（'a1'→0）
def record_to_idx(record):
  i = ord(record[0]) - ord('a')
  j = int(record[1])
  return i + (j-1) * 8

In [13]:
#確認用(tkinter)に最終盤面を保持したlistを作成
 
indexes_list = []
boards_list = [] #最終的に欲しいもの
for records in records_list:
  state = State() #初期化
  boards = []
  indexes = []

  #一試合分を作成
  boards.append(state.return_board())
  for record in records:
    index = record_to_idx(record)
    state = state.next(index)
    indexes.append(index)
    boards.append(state.return_board())

  #listにそれぞれ追加
  indexes_list.append(indexes)
  boards_list.append(boards)


ここからデータの微調整

In [14]:
#盤面の状態が1つ多いことを確認（最終盤面には対応する次の指手がない）
print(len(indexes_list), len(indexes_list[0]))
print(len(boards_list), len(boards_list[0]), len(boards_list[0][0]))

21518 60
21518 61 64


In [15]:
#学習用に最終盤面を削除, 学習用にリストの構造を変換、一回のみ実行すること
#後半の部分でreshapeみたいなことをしている（他の方法もあり）
boards_list_for_dataset = []
for boards in boards_list:
  boards.pop()
  for board in boards:
    boards_list_for_dataset.append(board)

In [17]:
print(len(boards_list_for_dataset),len(boards_list_for_dataset[0]))

1291080 64


In [18]:
#今回は不要（範囲外のデータを含めたデータをcsvに保存する場合必要）

　
#idxに関して
#list→pd.df→ndarrayにしてからreshape　(微妙)
#最後はpd.dfにしてcsv形式にできるようにする
# df_list_index = pd.DataFrame(indexes_list)
# np_index = df_list_index.to_numpy().reshape(-1,)
# df_index = pd.DataFrame(np_index)

#boardはrehapeに値するものはすでにしているのでpd.df形式にするのみ
# df_board = pd.DataFrame(boards_list_for_dataset)

## データ整形　（範囲外のものを削除）

In [27]:
df_list_index = pd.DataFrame(indexes_list)
label_list = df_list_index.to_numpy().reshape(-1).tolist()

In [29]:
#範囲内(0-64)の数値のインデックスリストを取得
true_idx = [i for i, num in enumerate(label_list) if num >= 0 & num <= 63]

In [32]:
#学習データの数が同じことを確認,正しいインデックスの数を確認
print(len(label_list), len(boards_list_for_dataset),len(true_idx)) 

1291080 1291080 1287359


In [33]:
#ここで正しい学習データのリストが取得できる
x =  [boards_list_for_dataset[i] for i in true_idx]
y =  [label_list[i] for i in true_idx]

In [34]:
df_x, df_y = pd.DataFrame(x), pd.DataFrame(y)

In [37]:
#csv化（trainとtestに分けていないもの）
df_x.to_csv('boards.csv')
df_y.to_csv('labels.csv')

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, train_size=0.8, random_state=0)

In [40]:
#csv化（trainとtestに分けたもの）
x_train.to_csv('train_boards.csv')
x_test.to_csv('test_boards.csv')
y_train.to_csv('train_labels.csv')
y_test.to_csv('test_labels.csv')

## テスト　確認

In [8]:
!python wthor_to_csv.py WTH_2020.wtb WTH_2019.wtb

In [9]:
dataset1 = pd.read_csv('/content/dataset_test.csv', index_col=0, header=0)
dataset1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,e6,f4,c3,c4,f3,d6,c5,e3,f6,f5,g5,c6,d3,b5,a6,a4,d7,c7,g4,d8,b8,e7,e8,c8,b4,a8,b7,f8,b6,c2,d2,a7,c1,a5,b3,a3,f7,g8,h8,a2,g6,h6,e1,e2,f1,f2,g7,h7,h5,h4,h3,h2,g3,g1,g2,h1,b2,a1,b1,d1
1,e6,f4,c3,c4,d3,c6,e3,f3,d6,c7,b4,c5,f5,g5,f6,d7,g6,g4,b5,e7,f7,f8,h5,h4,h3,g3,h6,e2,h2,b6,d2,f2,b8,d8,a6,a3,a5,d1,c2,b3,b1,c1,e1,g1,e8,c8,g8,a8,g7,a7,b7,a4,b2,a1,f1,a2,h1,g2,h7,h8
2,e6,f6,f5,d6,c5,e3,d7,f4,d3,e7,g5,c6,g4,c4,c7,d8,b4,h4,h6,h3,g6,h5,f8,h7,f7,f3,b6,c2,c8,e8,e2,c3,c1,b5,a5,b3,d2,a3,g3,h2,a4,a6,b7,a8,a7,b8,g7,h8,a2,g8,a1,b2,b1,d1,f2,g2,f1,e1,h1,g1
3,e6,f6,f5,d6,d7,g5,g6,e7,c5,c4,e8,f7,c6,d8,c7,e3,f4,h6,c8,b5,f3,d3,g4,b6,e2,h3,f8,f1,e1,f2,d2,g3,b3,b4,c3,d1,c1,c2,g1,a3,g2,g7,b1,b7,a8,b8,a5,h1,h8,g8,h7,a1,a2,a4,b2,h5,h4,h2,a7,a6
4,e6,f4,e3,f2,e2,f6,f5,d6,e7,f8,e8,d8,f3,f7,d7,g5,f1,c8,g6,h6,h5,h4,c7,g4,g3,d2,c6,d1,d3,b8,c1,b1,g7,h3,b7,h2,c5,h7,g2,c3,c4,h1,g1,e1,c2,a8,a7,a6,b5,b6,b4,a5,a4,a3,b3,a2,b2,a1,g8,h8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212,e6,f6,f5,d6,e7,g5,g6,f7,e8,f3,d7,h6,h4,c8,e3,h5,g4,h3,f4,f2,d2,g3,c3,f8,g8,c4,c6,d8,b8,b5,c7,c5,d3,c2,b4,b6,a5,c1,e2,d1,g2,b3,e1,a3,a4,a6,b1,h1,b2,f1,g1,a1,a2,b7,h2,g7,h7,h8,a7,a8
2213,e6,f4,c3,c4,d3,d6,f6,e7,f5,g6,g3,e3,c7,e2,c5,d7,c2,f3,g4,g5,f2,d2,c6,h4,f1,b5,a6,h3,e1,c8,b6,b4,b3,b2,f7,e8,a1,a4,a5,a7,g8,c1,b1,d1,a2,f8,g7,h8,h5,h6,d8,g2,h2,a3,a8,h1,g1,h7,b8,b7
2214,e6,f4,c3,c4,d3,d6,f6,c6,f5,g6,g4,g5,h6,e3,f3,d2,f7,e2,h5,c2,c5,g3,b4,b3,h2,f8,f2,a4,e1,d1,a3,a2,c1,f1,g1,b5,g7,b2,b1,h8,a5,a6,a1,b6,a7,h3,h4,h7,c7,c8,d8,e8,b7,e7,d7,h1,g2,a8,b8,g8
2215,e6,f4,c3,c4,d3,d6,f6,c6,f5,g5,f3,f7,h6,g6,e7,d7,c5,h4,c7,g4,d8,c8,b8,b6,e8,f8,g8,b5,h3,h2,b4,b3,b7,g3,g7,f2,f1,e2,e3,a8,h7,d2,a7,a6,e1,d1,c2,g1,a3,c1,b2,a5,a4,h8,h5,a1,h1,g2,b1,a2
